In [1]:
import sys
sys.path.insert(0, "..")
import sympy as sp
import sympy.physics.mechanics as me
import sympy.physics.vector as ve

# constants
g,c,k = sp.symbols('g c k')

# generalized coordinates, generalized speeds and their derivatives
q= me.dynamicsymbols('q1:11')
dq = me.dynamicsymbols('dq1:11', 1)
u = me.dynamicsymbols('u1:11')
du = me.dynamicsymbols('du1:11', 1)
t = sp.Symbol('t')

I_clavicula = sp.symbols('I_clavicula1:7')
I_scapula = sp.symbols('I_scapula1:7')
I_humerus = sp.symbols('I_humerus1:7')
I_ulna = sp.symbols('I_ulna1:7')
I_radius = sp.symbols('I_radius1:7')
I_hand = sp.symbols('I_hand1:7')

rigid0P = sp.symbols('rigid0P1:4')
rigid1P = sp.symbols('rigid1P1:4')
rigid1C = sp.symbols('rigid1C1:4')
rigid2P = sp.symbols('rigid2P1:4')
rigid2C = sp.symbols('rigid2C1:4')
rigid3C = sp.symbols('rigid3C1:4')
rigid3P = sp.symbols('rigid3P1:4')
rigid4C = sp.symbols('rigid4C1:4')
rigid4P = sp.symbols('rigid4P1:4')
rigid5C = sp.symbols('rigid5C1:4')
rigid5P = sp.symbols('rigid5P1:4')
rigid6C = sp.symbols('rigid6C1:4')

EL_P_offset_rot = sp.symbols('EL_P_offset_rot1:4')
EL_rot_axis = sp.symbols('EL_rot_axis1:4')
PSY_rot_axis = sp.symbols('PSY_rot_axis1:4')

# bodies
clavicula_com = me.Point('clavicula_com')
scapula_com = me.Point('scapula_com')
humerus_com = me.Point('humerus_com')
ulna_com = me.Point('ulna_com')
radius_com = me.Point('radius_com')
hand_com = me.Point('hand_com')

clavicula_frame = me.ReferenceFrame('clavicula_frame')
scapula_frame = me.ReferenceFrame('scapula_frame')
humerus_frame = me.ReferenceFrame('humerus_frame')
ulna_frame = me.ReferenceFrame('ulna_frame')
radius_frame = me.ReferenceFrame('radius_frame')
hand_frame = me.ReferenceFrame('hand_frame')


clavicula_mass, scapula_mass, humerus_mass, ulna_mass, radius_mass, hand_mass = sp.symbols('clavicula_mass scapula_mass humerus_mass ulna_mass radius_mass hand_mass')

clavicula_inertia = me.inertia(clavicula_frame, *I_clavicula)
scapula_inertia = me.inertia(scapula_frame, *I_scapula)
humerus_inertia = me.inertia(humerus_frame, *I_humerus)
ulna_inertia = me.inertia(ulna_frame, *I_ulna)
radius_inertia = me.inertia(radius_frame, *I_radius)
hand_inertia = me.inertia(hand_frame, *I_hand)

thorax = me.Body('Th')
clavicula = me.Body('Ul', clavicula_com, clavicula_mass, clavicula_frame, clavicula_inertia)
scapula = me.Body('Ul', scapula_com, scapula_mass, scapula_frame, scapula_inertia)
humerus = me.Body('Hu', humerus_com, humerus_mass, humerus_frame, humerus_inertia)
ulna = me.Body('Ul', ulna_com, ulna_mass, ulna_frame, ulna_inertia)
radius = me.Body('Ra', radius_com, radius_mass, radius_frame, radius_inertia)
hand = me.Body('Ha', hand_com, hand_mass, hand_frame, hand_inertia)

interframe1 = me.ReferenceFrame('interframe1')
interframe1.orient_body_fixed(humerus.frame,(0,0,EL_P_offset_rot[2]),'123')

# creating joints
SC = me.SphericalJoint('SC', thorax, clavicula, 
                        child_point=rigid1C[0]*clavicula.x+rigid1C[1]*clavicula.y+rigid1C[2]*clavicula.z,
                        parent_point=rigid0P[0]*thorax.x+rigid0P[1]*thorax.y+rigid0P[2]*thorax.z,
                        coordinates = [q[0], q[1], q[2]],
                        speeds = [u[0], u[1], u[2]],rot_order='231', rot_type = 'body')

AC = me.SphericalJoint('AC', clavicula,  scapula,  
                        child_point=rigid2C[0]*scapula.x+rigid2C[1]*scapula.y+rigid2C[2]*scapula.z,
                        parent_point=rigid1P[0]*clavicula.x+rigid1P[1]*clavicula.y+rigid1P[2]*clavicula.z, 
                        coordinates = [q[3], q[4], q[5]],
                        speeds = [u[3], u[4], u[5]],rot_order='231', rot_type = 'body')

GH = me.SphericalJoint('GH', scapula,  humerus,  
                        child_point=rigid3C[0]*humerus.x+rigid3C[1]*humerus.y+rigid3C[2]*humerus.z,
                        parent_point=rigid2P[0]*scapula.x+rigid2P[1]*scapula.y+rigid2P[2]*scapula.z,
                        coordinates = [q[6], q[7], q[8]],
                        speeds = [u[6], u[7], u[8]],rot_order='232', rot_type = 'body')

EL = me.PinJoint('EL',humerus,ulna,
                child_point = rigid4C[0]*ulna.x+rigid4C[1]*ulna.y+rigid4C[2]*ulna.z,
                parent_point = rigid3P[0]*humerus.x+rigid3P[1]*humerus.y+rigid3P[2]*humerus.z,
                parent_interframe = interframe1,
                joint_axis = EL_rot_axis[0]*interframe1.x+EL_rot_axis[1]*interframe1.y+EL_rot_axis[2]*interframe1.z,
                coordinates = q[9], speeds = u[9])

# PS = me.PinJoint('PS',ulna,radius,
PS = me.WeldJoint('PS',ulna,radius,
               child_point = rigid5C[0]*radius.x+rigid5C[1]*radius.y+rigid5C[2]*radius.z,
               parent_point = rigid4P[0]*ulna.x+rigid4P[1]*ulna.y+rigid4P[2]*ulna.z)
               # joint_axis = PSY_rot_axis[0]*ulna.x+PSY_rot_axis[1]*ulna.y+PSY_rot_axis[2]*ulna.z,
               # coordinates = q[10], speeds = u[10])

RC = me.WeldJoint('RC',radius,hand,
                  parent_point = rigid5P[0]*radius.x+rigid5P[1]*radius.y+rigid5P[2]*radius.z,
                  child_point = rigid6C[0]*hand.x+rigid6C[1]*hand.y+rigid6C[2]*hand.z)

# applying gravitational forces in Y direction
clavicula.apply_force(-thorax.y*clavicula.mass*g)
scapula.apply_force(-thorax.y*scapula.mass*g)
humerus.apply_force(-thorax.y*humerus.mass*g)
ulna.apply_force(-thorax.y*ulna.mass*g)
radius.apply_force(-thorax.y*radius.mass*g)   
hand.apply_force(-thorax.y*hand.mass*g)

# linear springs and dampings in joints
# angular velocities in base frame
w_clavicula = clavicula.ang_vel_in(thorax.frame)
w_scapula = scapula.ang_vel_in(clavicula.frame)
w_humerus = humerus.ang_vel_in(scapula.frame)

T1 = -c*w_clavicula.dot(clavicula.x)*clavicula.x
T2 = -c*w_clavicula.dot(clavicula.y)*clavicula.y
T3 = -c*w_clavicula.dot(clavicula.z)*clavicula.z

T4 = -c*w_scapula.dot(scapula.x)*scapula.x
T5 = -c*w_scapula.dot(scapula.y)*scapula.y
T6 = -c*w_scapula.dot(scapula.z)*scapula.z

T7 = -c*w_humerus.dot(humerus.x)*humerus.x
T8 = -c*w_humerus.dot(humerus.y)*humerus.y
T9 = -c*w_humerus.dot(humerus.z)*humerus.z

clavicula.apply_torque(T1+T2+T3,thorax)
scapula.apply_torque(T4+T5+T6,clavicula)
humerus.apply_torque(T7+T8+T9,scapula)

In [3]:
# Contact between scapula and thorax
contTS = sp.symbols('contTS1:4')
contAI = sp.symbols('contAI1:4')
elips_trans = sp.symbols('elips_trans1:4')
elips_dim = sp.symbols('elips_dim1:4')
k_contact_in, eps_in = sp.symbols('k_contact_in eps_in')
k_contact_out, eps_out = sp.symbols('k_contact_out eps_out')
second_elips_dim = sp.Symbol('second_elips_dim')

# contact points 
contact_point1 = me.Point('CP1')
contact_point1.set_pos(scapula.masscenter,contTS[0]*scapula.x+contTS[1]*scapula.y  +contTS[2]*scapula.z)
contact_point1.set_vel(scapula.frame,0) # point is fixed in scapula

contact_point2 = me.Point('CP2')
contact_point2.set_pos(scapula.masscenter,contAI[0]*scapula.x+contAI[1]*scapula.y  +contAI[2]*scapula.z)
contact_point2.set_vel(scapula.frame,0) # point is fixed in scapula

## contact forces

# Distances between contact points and thorax frame
x_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.x)+contTS[0]*scapula.x.dot(thorax.x)+contTS[1]*scapula.y.dot(thorax.x)+contTS[2]*scapula.z.dot(thorax.x)
y_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.y)+contTS[0]*scapula.x.dot(thorax.y)+contTS[1]*scapula.y.dot(thorax.y)+contTS[2]*scapula.z.dot(thorax.y)
z_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.z)+contTS[0]*scapula.x.dot(thorax.z)+contTS[1]*scapula.y.dot(thorax.z)+contTS[2]*scapula.z.dot(thorax.z)
x_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.x)+contAI[0]*scapula.x.dot(thorax.x)+contAI[1]*scapula.y.dot(thorax.x)+contAI[2]*scapula.z.dot(thorax.x)
y_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.y)+contAI[0]*scapula.x.dot(thorax.y)+contAI[1]*scapula.y.dot(thorax.y)+contAI[2]*scapula.z.dot(thorax.y)
z_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.z)+contAI[0]*scapula.x.dot(thorax.z)+contAI[1]*scapula.y.dot(thorax.z)+contAI[2]*scapula.z.dot(thorax.z)

# Contact forces
f1_in = ((x_pos1-elips_trans[0])/elips_dim[0])**2+((y_pos1-elips_trans[1])/elips_dim[1])**2+((z_pos1-elips_trans[2])/elips_dim[2])**2-1
f1_out = ((x_pos1-elips_trans[0])/(second_elips_dim*elips_dim[0]))**2+((y_pos1-elips_trans[1])/(second_elips_dim*elips_dim[1]))**2+((z_pos1-elips_trans[2])/(second_elips_dim*elips_dim[2]))**2-1
F1_in = 1/2*(f1_in-sp.sqrt(f1_in**2+eps_in**2))
F1_out = 1/2*(f1_out+sp.sqrt(f1_out**2+eps_out**2))
Fx1 = -(k_contact_in*F1_in+k_contact_out*F1_out)*(x_pos1-elips_trans[0])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[0]**2)
Fy1 = -(k_contact_in*F1_in+k_contact_out*F1_out)*(y_pos1-elips_trans[1])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[1]**2)
Fz1 = -(k_contact_in*F1_in+k_contact_out*F1_out)*(z_pos1-elips_trans[2])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[2]**2)

f2_in = ((x_pos2-elips_trans[0])/elips_dim[0])**2+((y_pos2-elips_trans[1])/elips_dim[1])**2+((z_pos2-elips_trans[2])/elips_dim[2])**2-1
f2_out = ((x_pos2-elips_trans[0])/(second_elips_dim*elips_dim[0]))**2+((y_pos2-elips_trans[1])/(second_elips_dim*elips_dim[1]))**2+((z_pos2-elips_trans[2])/(second_elips_dim*elips_dim[2]))**2-1
F2_in = 1/2*(f2_in-sp.sqrt(f2_in**2+eps_in**2))
F2_out = 1/2*(f2_out+sp.sqrt(f2_out**2+eps_out**2))
Fx2 = -(k_contact_in*F2_in+k_contact_out*F2_out)*(x_pos2-elips_trans[0])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[0]**2)
Fy2 = -(k_contact_in*F2_in+k_contact_out*F2_out)*(y_pos2-elips_trans[1])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[1]**2)
Fz2 = -(k_contact_in*F2_in+k_contact_out*F2_out)*(z_pos2-elips_trans[2])*(elips_dim[0]**2+elips_dim[1]**2+elips_dim[2]**2)/(elips_dim[2]**2)

# applying contact forces to contact points in thorax frame
scapula.apply_force(thorax.x*Fx1+thorax.y*Fy1+thorax.z*Fz1,contact_point1)
scapula.apply_force(thorax.x*Fx2+thorax.y*Fy2+thorax.z*Fz2,contact_point2)


In [4]:
# method for solving EoMs
method = me.JointsMethod(thorax,SC,AC,GH,EL,PS,RC)
method.form_eoms()
mm = method.mass_matrix_full
fo = method.forcing_full

In [5]:
## substituting coords (x1..x9) and speeds (x10..x18)
y = sp.symbols('y1:21')
subs_q = {q[i]: y[i] for i in range(len(q))}
subs_u = {u[i]: y[i+len(q)] for i in range(len(q))}
mm_subs = me.msubs(mm,subs_q,subs_u)
fo_subs = me.msubs(fo,subs_q,subs_u)

In [6]:
import sys
sys.path.insert(0, "..")
from importlib import reload
import pyMatlabFunction as pyMF
reload (pyMF)

const_vec = {
    'rigid0P': rigid0P,'rigid1C': rigid1C,'rigid1P': rigid1P,'rigid2C': rigid2C,'rigid2P': rigid2P,
    'rigid3C': rigid3C,'rigid3P': rigid3P,'rigid4C': rigid4C,'rigid4P': rigid4P,'rigid5C': rigid5C,
    'rigid5P': rigid5P,'rigid6C': rigid6C,
    'EL_P_offset_rot': EL_P_offset_rot,'EL_rot_axis': EL_rot_axis,'PSY_rot_axis': PSY_rot_axis,
    'I_clavicula': I_clavicula,'I_scapula': I_scapula,'I_humerus': I_humerus,'I_ulna': I_ulna,
    'I_radius': I_radius,'I_hand': I_hand,
    'contTS': contTS,
    'contAI': contAI,
    'elips_trans':elips_trans,
    'elips_dim':elips_dim
}
# 
const = {
    'c': c,'g': g,'k_contact_in': k_contact_in,'eps_in': eps_in,
    'k_contact_out': k_contact_out,'eps_out': eps_out, 'second_elips_dim': second_elips_dim,
    'clavicula_mass': clavicula_mass,'scapula_mass': scapula_mass,
    'radius_mass': radius_mass,'humerus_mass': humerus_mass,
    'ulna_mass': ulna_mass,'hand_mass': hand_mass,
}

pyMF.MatlabFunction(function = fo_subs,
                    fun_name = 'fo_py_ulna_InOutCont',assignto = 'fo',
                    states = y,constants = const,constants_vec = const_vec)

pyMF.MatlabFunction(function = mm_subs,
                    fun_name = 'mm_py_ulna_InOutCont',assignto = 'mm',
                    states = y,constants = const,constants_vec = const_vec)